# Interview Task – Data Engineering & Analytics

## 1. Datasets Issues and Requirements

In [1]:
# imports
import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import scipy.stats as stats

### Import the datasets

In [2]:
df = pd.read_csv("aviation_data.csv")
df.head()

,FlightNumber,DepartureDate,DepartureTime,ArrivalDate,ArrivalTime,Airline,DelayMinutes
0,AA1234,09/01/2023,08:30 AM,09/01/2023,10:45 AM,American Airlines,15.0
1,DL5678,09/01/2023,01:15 PM,09/01/2023,03:30 PM,Delta,5.0
2,UA9101,09/01/2023,05:00 PM,09/01/2023,07:15 PM,United Airlines,25.0
3,AA1234,09/01/2023,08:30 AM,09/01/2023,10:45 PM,American Airlines,30.0
4,DL5678,09/02/2023,02:00 PM,09/02/2023,04:10 PM,Delta,NaN


### Insert the datasets into a MySQL database

In [3]:
from sqlalchemy import create_engine, text
import os
import pandas as pd
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Retrieve database credentials from environment variables
DB_USERNAME = os.getenv("DB_USERNAME")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_PORT = os.getenv("DB_PORT")


connection_string = f"mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    engine = create_engine(connection_string)
    # create table
    create_table_query = text("""
    CREATE TABLE IF NOT EXISTS aviation_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        FlightNumber TEXT,
        DepartureDate TEXT,
        DepartureTime TEXT,
        ArrivalDate TEXT,
        ArrivalTime TEXT,
        Airline TEXT,
        DelayMinutes FLOAT
    )"""
    )

    with engine.connect() as connection:
        connection.execute(create_table_query)

    df.to_sql("aviation_data", engine, if_exists="append", index=False)

    df_fetched = pd.read_sql("SELECT * FROM aviation_data", engine)

    print(df_fetched.head())

except Exception as e:
    print(f"Error: {e}")

finally:
    engine.dispose()

   id FlightNumber DepartureDate DepartureTime ArrivalDate ArrivalTime  \
0   1       AA1234    09/01/2023      08:30 AM  09/01/2023    10:45 AM   
1   2       DL5678    09/01/2023      01:15 PM  09/01/2023    03:30 PM   
2   3       UA9101    09/01/2023      05:00 PM  09/01/2023    07:15 PM   
3   4       AA1234    09/01/2023      08:30 AM  09/01/2023    10:45 PM   
4   5       DL5678    09/02/2023      02:00 PM  09/02/2023    04:10 PM   

             Airline  DelayMinutes  
0  American Airlines          15.0  
1              Delta           5.0  
2    United Airlines          25.0  
3  American Airlines          30.0  
4              Delta           NaN  


In [4]:
# DepartureDate and ArrivalDate are in MM/DD/YYYY format.
